In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gabe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Gabe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Gabe\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Gabe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Gabe\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
reviews_df = pd.read_csv("bgg-19m-reviews.csv")
# dropping extra index column and rows without comments
reviews_df = reviews_df.dropna(subset=['comment'])
reviews_df = reviews_df.drop(axis=1, columns="Unnamed: 0")
# creating sentiment column based on mean scores found in EDA
reviews_df.sample(10)

,user,rating,comment,ID,name
16636921,Superfage,6.50,A good mix of german and war game! Yuo need to...,870,Empires of the Ancient World
11102670,jollster725,6.70,Experience (35%) - 3/5 Mechanics (20%) - 4/5 I...,241724,Villagers
1815617,DancerInDC,8.75,What I like about this game is its simplicity ...,822,Carcassonne
17608509,jonny morbid,5.00,Another game based on ALIEN or--if you're a RE...,2613,Intruder
5654816,Jedi Mace,7.00,Very good Partygame. It's short and makes a lo...,147949,One Night Ultimate Werewolf
11944476,bwingrave,4.00,Negotiation game involving nasty card play. D...,115,I'm the Boss!
15259994,Kite Eating Tree,6.00,When this game came out I was addicted. It was...,798,Ace of Aces: Handy Rotary Series
16700825,jrigkvn1229,3.00,Extra! has much better risk reward,217547,Can't Stop Express
12744235,the_horror,8.00,Great introduction to the COIN series.,111799,Cuba Libre
17390180,David Ells,8.00,Kickstarter!,139572,Karesansui


In [3]:
board_games_df = pd.read_csv("games_detailed_info.csv")
# dropping extra index column and comlumns that are outside of the scope of the model, such as summary statistics and alternate rankings.
board_games_df = board_games_df.drop(axis=1, columns="Unnamed: 0")
board_games_df = board_games_df.drop(columns=[ 'thumbnail', 'boardgameexpansion', 'boardgameimplementation', 'bayesaverage','Strategy Game Rank', 'Family Game Rank', 'stddev', 'median', 'numweights',
       'averageweight', 'boardgameintegration', 'boardgamecompilation',
       'Party Game Rank', 'Abstract Game Rank', 'Thematic Rank',
       'War Game Rank', 'Customizable Rank', "Children's Game Rank",
       'RPG Item Rank', 'Accessory Rank', 'Video Game Rank', 'Amiga Rank',
       'Commodore 64 Rank', 'Arcade Rank', 'Atari ST Rank', 'suggested_num_players', 'image'])
board_games_df.sample(10)

C:\Users\Gabe\AppData\Local\Temp\ipykernel_23780\3871456848.py:1: DtypeWarning: Columns (29,48) have mixed types. Specify dtype option on import or set low_memory=False.
  board_games_df = pd.read_csv("games_detailed_info.csv")


,type,id,primary,alternate,description,yearpublished,minplayers,maxplayers,suggested_playerage,suggested_language_dependence,...,boardgameartist,boardgamepublisher,usersrated,average,Board Game Rank,owned,trading,wanting,wishing,numcomments
8509,boardgame,62814,Tumblin-Dice Medium,NaN,A dexterity dice game consisting of four sets ...,2008,2,6,"[OrderedDict([('@value', '2'), ('@numvotes', '...","[OrderedDict([('@level', '6'), ('@value', 'No ...",...,NaN,['Ferti'],189,7.24975,5436,291,6,7,35,48
8824,boardgame,754,Domain,"['2001: The space age game', 'Boomerang', 'Cha...",Domain is a simple game played on a 9x9 square...,1982,2,2,"[OrderedDict([('@value', '2'), ('@numvotes', '...","[OrderedDict([('@level', '1'), ('@value', 'No ...",...,NaN,"['Alga', 'Editrice Giochi', 'Estrela', 'Nathan...",178,6.12156,10530,396,33,6,28,92
10827,boardgame,170909,Caçadores da Galáxia,NaN,"In a distant future, tiranic megacorporations ...",2015,2,4,"[OrderedDict([('@value', '2'), ('@numvotes', '...","[OrderedDict([('@level', '16'), ('@value', 'No...",...,"['Xia Taptara', 'Cássio Yoshiaki']","['HISTERIA GAMES', 'Taberna do Dragão']",124,7.68710,6709,214,10,2,25,19
12702,boardgame,708,Holy War: Afghanistan,NaN,&quot;Holy War: Afghanistan&quot; is a two-pla...,1991,2,2,"[OrderedDict([('@value', '2'), ('@numvotes', '...","[OrderedDict([('@level', '46'), ('@value', 'No...",...,['Mark Simonitch'],['Decision Games (I)'],91,6.23187,12020,495,26,12,29,74
17121,boardgame,170423,Comment j'ai adopté un gnou,['Cómo Adopté un Ñu'],The active player chooses a random theme by th...,2014,3,8,"[OrderedDict([('@value', '2'), ('@numvotes', '...","[OrderedDict([('@level', '6'), ('@value', 'No ...",...,['Fabien Bleuze'],"['Asmodee Spain', 'Le Droit de Perdre', 'LongA...",50,6.24080,14226,116,4,1,3,11
13146,boardgame,169045,Star Wars Galactic Dice Game,NaN,"In this Star Wars dice game, take your points ...",2014,2,8,"[OrderedDict([('@value', '2'), ('@numvotes', '...","[OrderedDict([('@level', '36'), ('@value', 'No...",...,NaN,['Cardinal'],85,5.28706,19271,301,4,0,10,30
19356,boardgame,253564,Marvel Dice Masters: X-Men Forever Campaign Box,NaN,The Marvel Dice Masters: X-Men Forever Campaig...,2019,2,2,"[OrderedDict([('@value', '2'), ('@numvotes', '...","[OrderedDict([('@level', '71'), ('@value', 'No...",...,NaN,['WizKids'],38,7.51061,11367,146,1,18,33,7
10869,boardgame,2002,Magical Maze,"[""Goblin's Gold"", ""Mag'nus et la pièce d'or"", ...","With the aid of the Wizard, you'll try to get ...",1994,2,4,"[OrderedDict([('@value', '2'), ('@numvotes', '...","[OrderedDict([('@level', '11'), ('@value', 'No...",...,NaN,"['Jumbo', 'Nathan', 'University Games']",123,5.31390,20021,262,14,5,7,54
18624,boardgame,160406,Khrysos Hunters,NaN,Mysterious treasures have been found in the Kh...,2014,2,4,"[OrderedDict([('@value', '2'), ('@numvotes', '...","[OrderedDict([('@level', '1'), ('@value', 'No ...",...,['Josh Rivers'],['The Flux Capacity'],42,5.61429,18326,135,8,0,6,19
17768,boardgame,6665,Legend of Camelot,NaN,From the back of the box:&#10;&quot;Relive the...,1987,2,4,"[OrderedDict([('@value', '2'), ('@numvotes', '...","[OrderedDict([('@level', '51'), ('@value', 'No...",...,NaN,['Hoyle Products'],46,5.70217,17140,124,12,6,19,26


### Recomendations based on user review scores using kNN

In [4]:
# going to be using thesee to create the user review matrix dimensions
n_ratings = len(reviews_df)
n_games = len(reviews_df['ID'].unique())
n_users = len(reviews_df['user'].unique())
print(n_ratings, n_games, n_users)

3368619 21828 162324


In [5]:
#maps idices to users and game IDs

user_map = dict(zip(np.unique(reviews_df["user"]), list(range(n_users))))
game_map = dict(zip(np.unique(reviews_df["ID"]), list(range(n_games))))
print(len(user_map) == n_users)
print(len(game_map) == n_games)

user_i_map = dict(zip(list(range(n_users)), np.unique(reviews_df["user"])))
game_i_map = dict(zip(list(range(n_games)), np.unique(reviews_df["ID"])))

True
True


In [6]:
# creating indices for csr_matrix
user_index = [user_map[i] for i in reviews_df['user']]
game_index = [game_map[i] for i in reviews_df['ID']]



In [7]:
# creates csr matrix
matrix = csr_matrix((reviews_df["rating"], (game_index, user_index)), shape=(n_games, n_users))
matrix

<21828x162324 sparse matrix of type '<class 'numpy.float64'>'
	with 3368619 stored elements in Compressed Sparse Row format>

In [23]:
def find_related_games(game_id, k):
    # going to store the IDs for nearest games
    neighbour_ids = []
    
    # Honestly kinda confused as to what's going on here. I think the vectors are the euclidean distances of the other games in relation to the movie_id
    game_index = game_map[game_id]
    game_vector = matrix[game_index]
    k+=1
    # knn model fits created matrix so the 
    kNN = NearestNeighbors(algorithm = 'brute', metric='cosine')
    kNN.fit(matrix)
    game_vec = game_vector.reshape(1,-1)
    # uses kneighborst to fin the 'k' most related games
    distances, indices = kNN.kneighbors(game_vec, n_neighbors=k)
    # adds returned list of games to neighbor_ids
    for i in range(0,len(distances.flatten())):
        n = indices.flatten()[i]
        neighbour_ids.append(game_i_map[n])
    neighbour_ids.pop(0)
    return neighbour_ids
  

In [9]:
# creating a dictionary to pull the name of games from based on IDs
game_names = dict(zip(reviews_df['ID'], reviews_df['name']))

In [10]:
game_id = 151247
  
recommended_games = find_related_games(game_id, k=10)
  
print(f"Games related to: {game_names[game_id]}")
for i in recommended_games:
    print(game_names[i])

Games related to: Greed
Deus
Copycat
Elysium
Kashgar: Merchants of the Silk Road
The Palaces of Carrara
Rise of Augustus
Spyrium
Bruges
Temporum
Prosperity


In [11]:
game_id = 9209
  
recommended_games = find_related_games(game_id, k=10)
  
print(f"Games related to: {game_names[game_id]}")
for i in recommended_games:
    print(game_names[i])

Games related to: Ticket to Ride
Carcassonne
Catan
Pandemic
Lost Cities
Puerto Rico
Dominion
Power Grid
San Juan
Small World
Alhambra


In [12]:
game_id = 28023
  
recommended_games = find_related_games(game_id, k=10)
  
print(f"Games related to: {game_names[game_id]}")
for i in recommended_games:
    print(game_names[i])
    

Games related to: Jamaica
King of Tokyo
For Sale
Stone Age
No Thanks!
Diamant
Tobago
Biblios
Kingsburg
Jaipur
Takenoko


### Using game descriptions

In [13]:
# created a DF with IDs and game descriptions and set IDs as the index
game_desc_df = board_games_df[["id", "description"]]
print(type(game_desc_df))
game_desc_df.columns = ['ID', 'Description']
game_desc_df = game_desc_df.set_index('ID')



game_desc_df.sample(10)

<class 'pandas.core.frame.DataFrame'>


,Description
ID,
206448,"Welcome to Noxford, a timeless city in perpetu..."
316113,Buy the Vote is a 15 minute blind bidding game...
22614,This is a simple economic game about oil compa...
35035,Blood and Bridges is a complete stand-alone Wo...
104798,Game description from the publisher:&#10;&#10;...
4519,A version of the traditional game &quot;Yacht&...
191432,Town of Salem is a popular online game that go...
266938,Come along on the Smash Up World Tour!&#10;&#1...
312767,Astoria is a land pulsing with magic and intri...


In [14]:
# variables to be used in the sentance processing
lemmatizer = WordNetLemmatizer()

stop_words = set(stopwords.words('english'))
  
VERB_CODES = {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'}

In [15]:

def process_sentence(text):
    # changing to lower case, tokenizing, and lemmatizing each fo teh descriptions
    text = str(text).lower()
    temp_sent =[]
    words = nltk.word_tokenize(text)
    tags = nltk.pos_tag(words)
    for i, word in enumerate(words):
        if tags[i][1] in VERB_CODES: 
            lemmatized = lemmatizer.lemmatize(word, 'v')
        else:
            lemmatized = lemmatizer.lemmatize(word)
        if lemmatized not in stop_words and lemmatized.isalpha():
            temp_sent.append(lemmatized)
    # joining tokenized words and removing contractions
    finalsent = ' '.join(temp_sent)
    finalsent = finalsent.replace("n't", " not")
    finalsent = finalsent.replace("'m", " am")
    finalsent = finalsent.replace("'s", " is")
    finalsent = finalsent.replace("'re", " are")
    finalsent = finalsent.replace("'ll", " will")
    finalsent = finalsent.replace("'ve", " have")
    finalsent = finalsent.replace("'d", " would")
    return finalsent


In [16]:
# cleaning each of the descriptions
game_desc_df['Processed_Description'] = game_desc_df.Description.apply(process_sentence)
game_desc_df.sample(10)

,Description,Processed_Description
ID,,
194505,"Tallinn (Reval, Castrum Danorum) in late 14th ...",tallinn reval castrum danorum late century try...
261355,Cortex 3 is another member of the Cortex famil...,cortex another member cortex family popular ga...
23346,The first cardgame from award winning designer...,first cardgame award win designer han van tol ...
73365,Get rid of your Payoo!&#10;&#10;There are no j...,get rid payoo jack queen king papayoo ndash un...
128011,Extra! Extra! is all about completing the fron...,extra extra complete front back page newspaper...
519,This dice-heavy racing game is set in the grea...,racing game set great white north alaska effic...
873,Briefly sold as at Essen 2000 as &quot;Online....,briefly sell essen quot quot tile laying game ...
28143,2018 UPDATE: The second edition of the game is...,update second edition game improve cvd color b...
108012,"April 14, 1912, 11:40 p.m.: In the dark, cold ...",april dark cold water atlantic ocean transatla...


In [17]:
# saving to CSV just in case
game_desc_df.to_csv("game_desc_df.csv")

In [18]:
# creating and applying TFIDF vectorizer to game descriptions and using cosine similarities to find the closest matches
tfidfvec = TfidfVectorizer()

tfidf_gamedesc = tfidfvec.fit_transform((game_desc_df['Processed_Description']))

cos_sim = cosine_similarity(tfidf_gamedesc, tfidf_gamedesc)

In [19]:
# functions for retrieving similar game IDs from cos_sim
indices = pd.Series(game_desc_df.index)
  
def get_recommendations(id):
    recommended_games = []
    index = indices[indices == id].index[0]
    similarity_scores = pd.Series(cos_sim[index]).sort_values(ascending = False)
    top = list(similarity_scores.iloc[1:11].index)
    for i in top:
        recommended_games.append(list(game_desc_df.index)[i])
    return recommended_games

In [20]:
game_id = 151247
  
recommended_games = get_recommendations(game_id)
  
print(f"Games related to: {game_names[game_id]}")
for i in recommended_games:
    print(game_names[i])

Games related to: Greed
Ohanami
Escalation!
Vantage
Zwickern
Mū
Pets
Naova
6 nimmt! 25 Jahre
Lao Pengh
The Bark Side


In [21]:
game_id = 9209
  
recommended_games = get_recommendations(game_id)
  
print(f"Games related to: {game_names[game_id]}")
for i in recommended_games:
    print(game_names[i])

Games related to: Ticket to Ride
Ticket to Ride: First Journey (Europe)
Ticket to Ride: Europe
Ticket to Ride: First Journey (U.S.)
Ticket to Ride: The Card Game
Ticket to Ride: New York
Ticket to Ride: Rails & Sails
Ticket to Ride: London
Zug um Zug: Deutschland
Ticket to Ride: Amsterdam
Ticket to Ride: Germany


In [22]:
game_id = 28023
  
recommended_games = get_recommendations(game_id)
  
print(f"Games related to: {game_names[game_id]}")
for i in recommended_games:
    print(game_names[i])

Games related to: Jamaica
Myths and Legends
Master Merchant
Emmerlaus Duel of Mages
James Clavell's Shogun Card Game
Fighting Fantasy Battle Cards
Monad
Jolly & Roger
Top Race
Monopoly: Klingon Edition
Bad Bunnies
